# Libraries

In [ ]:
import kagglehub
import pandas as pd
import numpy as np

path = kagglehub.dataset_download("joniarroba/noshowappointments")
print(f"Dataset downloaded to: {path}")

100%|██████████| 2.40M/2.40M [00:00<00:00, 65.8MB/s]

Extracting files...
Dataset downloaded to: /root/.cache/kagglehub/datasets/joniarroba/noshowappointments/versions/5


# Data Loading

In [ ]:
df = pd.read_csv(f"{path}/KaggleV2-May-2016.csv")

In [ ]:
df.head()

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,2.987250e+13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,5.589978e+14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4.262962e+12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,8.679512e+11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
4,8.841186e+12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,No


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110527 entries, 0 to 110526
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   PatientId       110527 non-null  float64
 1   AppointmentID   110527 non-null  int64  
 2   Gender          110527 non-null  object 
 3   ScheduledDay    110527 non-null  object 
 4   AppointmentDay  110527 non-null  object 
 5   Age             110527 non-null  int64  
 6   Neighbourhood   110527 non-null  object 
 7   Scholarship     110527 non-null  int64  
 8   Hipertension    110527 non-null  int64  
 9   Diabetes        110527 non-null  int64  
 10  Alcoholism      110527 non-null  int64  
 11  Handcap         110527 non-null  int64  
 12  SMS_received    110527 non-null  int64  
 13  No-show         110527 non-null  object 
dtypes: float64(1), int64(8), object(5)
memory usage: 11.8+ MB


In [ ]:
df.describe()

,patientid,appointmentid,age,scholarship,hipertension,diabetes,alcoholism,handcap,sms_received,no_show
count,1.105210e+05,1.105210e+05,110521.000000,110521.000000,110521.000000,110521.000000,110521.000000,110521.000000,110521.000000,110521.000000
mean,1.474921e+14,5.675304e+06,37.085694,0.098271,0.197248,0.071869,0.030401,0.022213,0.321034,0.201916
std,2.560928e+14,7.129576e+04,23.104606,0.297682,0.397923,0.258272,0.171690,0.161440,0.466876,0.401432
min,3.921784e+04,5.030230e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.172457e+12,5.640285e+06,18.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.172598e+13,5.680569e+06,37.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,9.438963e+13,5.725523e+06,55.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
max,9.999816e+14,5.790484e+06,102.000000,1.000000,1.000000,1.000000,1.000000,4.000000,1.000000,1.000000


# Data Preprocessing

In [ ]:
df.columns = df.columns.str.lower().str.replace('-', '_')

df['scheduledday'] = pd.to_datetime(df['scheduledday'])
df['appointmentday'] = pd.to_datetime(df['appointmentday'])

df['no_show'] = df['no_show'].map({'Yes': 1, 'No': 0})

df = df[(df['age'] >= 0) & (df['age'] <= 110)]

# Feature Engineering

In [ ]:
['waiting_days'] = (df['appointmentday'] - df['scheduledday']).dt.days
df['appointment_dow'] = df['appointmentday'].dt.dayofweek
df['scheduled_hour'] = df['scheduledday'].dt.hour

df['prev_no_shows'] = df.groupby('patientid')['no_show'].shift().fillna(0)
df['appointment_count'] = df.groupby('patientid').cumcount() + 1

conditions = ['hipertension', 'diabetes', 'alcoholism', 'handcap']
df['condition_count'] = df[conditions].sum(axis=1)

In [ ]:
neighborhood_stats = df.groupby('neighbourhood')['no_show'].agg(['mean', 'count'])
df = df.merge(neighborhood_stats, on='neighbourhood', suffixes=('', '_neighborhood'))

df['days_until_weekend'] = (4 - df['appointment_dow']) % 7
df['morning_appointment'] = (df['scheduled_hour'] < 12).astype(int)

In [ ]:
df.drop(['patientid', 'appointmentid'], axis=1, inplace=True)

df = pd.get_dummies(df, columns=['gender', 'neighbourhood'])

df.to_csv('processed_no_shows.csv', index=False)